In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv('amazon-fine-food-reviews/Reviews.csv')

In [3]:
reviews_ss = reviews[['Score','Text']]

In [4]:
sample_data = reviews_ss[reviews_ss.Score != 3].sample(1000)

In [5]:
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 117632 to 51427
Data columns (total 2 columns):
Score    1000 non-null int64
Text     1000 non-null object
dtypes: int64(1), object(1)
memory usage: 23.4+ KB


In [6]:
sample_data.Score.value_counts()

5    686
4    140
1    112
2     62
Name: Score, dtype: int64

In [7]:
def f(r):
    if r > 3:
        return 1
    else:
        return 0
    
sample_data['Sentiment'] = sample_data.Score.map(f)

In [8]:
sample_data.head()

,Score,Text,Sentiment
117632,5,"Bright, shiny colors give my projects POP! Te...",1
145531,4,I've been looking for a really good peachy fla...,1
189295,4,"Given this blend's orgin, the name ""English"" B...",1
338610,5,I'm not a morning person and I need my morning...,1
249372,5,These ginger snaps are GREAT! They taste and l...,1


In [9]:
from keras.preprocessing.text import Tokenizer

/home/awantik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
sample_data.Sentiment.value_counts()

1    826
0    174
Name: Sentiment, dtype: int64

In [11]:
max_features = 2000
tok = Tokenizer(num_words=max_features,split= ' ')

In [12]:
tok.fit_on_texts(sample_data.Text.values)

In [55]:
#tok.fit_on_sequences(X)

In [13]:
X = tok.texts_to_sequences(sample_data.Text.values)

In [14]:
X[0]

[1112,
 1411,
 136,
 13,
 644,
 234,
 8,
 23,
 186,
 18,
 1628,
 4,
 133,
 7,
 923,
 2,
 90,
 267,
 5,
 92,
 208,
 675,
 1,
 1011,
 309,
 1,
 1629,
 25,
 645,
 2,
 67,
 166,
 5,
 1916,
 13,
 1629,
 134,
 2,
 166,
 5,
 1630,
 4,
 304,
 1012,
 35,
 30,
 69,
 2,
 532,
 13,
 122,
 4,
 805,
 7,
 1240,
 3,
 1631,
 209,
 1,
 1629,
 43,
 618,
 2,
 1,
 1240,
 3,
 763,
 1,
 1629,
 220,
 11,
 1,
 21,
 67,
 237,
 4,
 133]

In [15]:
#tok.word_index

In [16]:
#tok.word_index

In [17]:
from keras.preprocessing.sequence import pad_sequences

In [18]:
X_pad = pad_sequences(X)

In [19]:
X_pad.shape

(1000, 575)

In [20]:
X_pad

array([[  0,   0,   0, ..., 237,   4, 133],
       [  0,   0,   0, ...,   4,   3, 437],
       [  0,   0,   0, ..., 183,   7,  39],
       ...,
       [  0,   0,   0, ...,  30, 162,   6],
       [  0,   0,   0, ...,  52,  31, 192],
       [  0,   0,   0, ...,  38, 731, 349]], dtype=int32)

In [27]:
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM
from keras.layers.core import SpatialDropout1D

In [39]:
model = Sequential()

In [40]:
model.add(Embedding(max_features, 128, input_length=X_pad.shape[1]))

In [41]:
model.add(SpatialDropout1D(0.4))

In [42]:
model.add(LSTM(196,dropout=0.2))

In [43]:
model.add(Dense(2,activation='softmax'))

In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 575, 128)          256000    
_________________________________________________________________
spatial_dropout1d_3 (Spatial (None, 575, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________


In [46]:
from sklearn.model_selection import train_test_split

In [51]:
Y = pd.get_dummies(sample_data.Sentiment).values

In [52]:
trainX,testX, trainY,testY = train_test_split(X_pad,Y)

In [54]:
model.fit(trainX,trainY, epochs=10, batch_size=128,validation_split=0.2)

Train on 600 samples, validate on 150 samples
Epoch 1/10
600/600 [==============================] - 27s 46ms/step - loss: 0.4086 - acc: 0.8333 - val_loss: 0.4172 - val_acc: 0.8200
Epoch 2/10
600/600 [==============================] - 29s 49ms/step - loss: 0.3849 - acc: 0.8333 - val_loss: 0.3913 - val_acc: 0.8200
Epoch 3/10
600/600 [==============================] - 26s 44ms/step - loss: 0.3433 - acc: 0.8333 - val_loss: 0.3579 - val_acc: 0.8200
Epoch 4/10
600/600 [==============================] - 26s 43ms/step - loss: 0.2840 - acc: 0.8433 - val_loss: 1.1381 - val_acc: 0.8200
Epoch 5/10
600/600 [==============================] - 24s 40ms/step - loss: 0.3770 - acc: 0.8850 - val_loss: 0.3788 - val_acc: 0.8867
Epoch 6/10
600/600 [==============================] - 24s 40ms/step - loss: 0.3316 - acc: 0.9367 - val_loss: 0.3506 - val_acc: 0.8800
Epoch 7/10
600/600 [==============================] - 28s 46ms/step - loss: 0.2573 - acc: 0.9367 - val_loss: 0.3010 - val_acc: 0.8733
Epoch 8/10
600/6

In [56]:
model.predict_classes(X_pad[:2])

array([1, 1])

In [71]:
d = tok.texts_to_sequences(['The food was awesome','I hate'])

In [72]:
X_t = pad_sequences(d,maxlen=575)

In [73]:
model.predict_classes(X_t)

array([1, 1])